## Feature Selection for Image data
### Introduction
Images are a rich source of information, with each pixel (and the relationships between pixels) contributing to the overall meaning. However, this can also make image data extremely high-dimensional. In practical terms, this means we have a vast number of features (pixels) to process, which can be both computationally expensive and prone to overfitting if not handled carefully.

In this notebook, we will explore various techniques for identifying and retaining the most important features (or representations) in image data. By focusing on these features, we can often build more efficient and accurate models.

### Why conventional feature selection methods may not suffice

Conventional methods such as variance thresholding, chi-squared tests, or straightforward recursive feature elimination are sometimes effective for numeric or text data. These methods often assume that features are largely independent from one another. In images, however, neighbouring pixels are usually correlated, meaning that the importance of one pixel can depend strongly on nearby pixels. Removing individual pixels based solely on variance or mutual correlation can fail to capture the distinctive patterns that frequently matter in images.

In practice, the techniques we use for image data try to find higher-level patterns or important regions. For example, a few pixels that form the eye of a cat or dog might matter more than background areas of the image, and these pixels are best considered together.

Below, we examine different ways to reduce the dimensionality or extract key features from images, ranging from classical methods (like entropy or mutual information) to more modern deep learning approaches (such as pre-trained convolutional neural networks or saliency-based methods).

### Installing Python libraries

In [ ]:
!pip install --upgrade pip

!pip install pandas numpy matplotlib opencv-python Pillow tensorflow scikit-image

### Download the image data
Let us assume we have two categories of images, *cats* and *dogs*, and that our dataset is stored in a directory `./cats_dogs/train/`, which contains two subdirectories `cats/` and `dogs/` where each folder holds multiple images of the respective animal:

```
./cats_dogs/train/
    cats/
      cat_1.jpg
      cat_2.jpg
      ...
    dogs/
      dog_1.jpg
      dog_2.jpg
      ...
```

We will resize each image to 64x64 pixels, then flatten them into one-dimensional vectors for initial processing. Each element in the flattened array corresponds to a single pixel intensity (for colour images, we have three channels: red, green, and blue, each contributing one dimension per pixel).

In [ ]:
import os
import urllib.request
import zipfile

# Define URL and target filenames
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_path = "cats_and_dogs_filtered.zip"
extract_dir = "cats_dogs"

# Download the zip file
print("Downloading dataset...")
urllib.request.urlretrieve(url, zip_path)
print("Download complete.")

# Extract the zip file
print("Extracting files...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Extraction complete.")

# Delete the zip file
print("Cleaning up...")
os.remove(zip_path)
print("Cleanup complete.")


### Load the image data

In [ ]:
import cv2
import numpy as np
import os
import glob

# Set a fixed image size for resizing
IMG_SIZE = 64

dataset_path = "cats_dogs/cats_and_dogs_filtered/train/" # Path to the folder containing data

category_path = ["cats", "dogs"]  # The subfolders - class labels

# Lists to hold the image data and labels
X = []  # Features
Y = []  # Labels (0 for cats, 1 for dogs)

limit = 100 # Limit the number of images to load per category to speed things up

for label, category in enumerate(category_path):
    path = os.path.join(dataset_path, category)
    for img_name in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_name))
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        if len(X) >= limit:
            break

        X.append(img)
        Y.append(label)

X = np.array(X, dtype=np.float32) / 255.0 # normalise
Y = np.array(Y)

print(f'Loaded {len(X)} images')
print(f'Each image is resized to {IMG_SIZE}x{IMG_SIZE} pixels')
print(f'Shape of X: {X.shape}')
print(f'Shape of Y: {Y.shape}')

### Feature selection techniques for images
When working with image data, one of the biggest challenges is its *high dimensionality* — each image contains thousands or even millions of pixels. Not all of this information is useful for making predictions, so we use *feature selection* techniques to focus only on the most important parts of the image.

We will introduce a variety of methods for identifying or extracting the most relevant features from images:

- *Feature Extraction using Pre-Trained CNNs*: These models, trained on large image datasets, can automatically detect patterns like edges, shapes, or textures that are helpful for classification.

- *Saliency Maps and Grad-CAM*: These techniques highlight which parts of an image had the biggest influence on the model’s decision, helping us understand what the model is “looking at.”

- *Entropy-Based Feature Selection*: Entropy measures the complexity or texture in different parts of an image, allowing us to focus on regions with more informative content.

- *Mutual Information*: This is a statistical method that looks at how much each pixel or region contributes to predicting the target label.

- *Dimensionality Reduction with Autoencoders*: Autoencoders are a type of neural network that learn to compress image data into a smaller, more useful representation, capturing only the most important information.

Each of these approaches reduces the amount of data we need to work with, making models faster and more efficient — while still preserving the key information needed for accurate predictions. Some rely on deep learning to automatically learn important patterns, while others use statistical techniques to rank and select features.

### Feature extraction using pre-trained CNNs

A powerful and popular way to handle image data is to harness the power of a pre-trained convolutional neural network (CNN). Well-known networks like VGG16, ResNet, or MobileNet have been trained on massive datasets (such as ImageNet), allowing them to learn general-purpose representations of images.

Instead of training a model from scratch on our smaller cat-and-dog dataset, we can use the internal layers of these pre-trained networks to extract discriminative features. This technique often provides a substantial boost in performance, even if the images are not from the exact same domain as the original training data.

Below is an example using VGG16, where we:

- load VGG16 (pre-trained on ImageNet) up to its last convolutional layer (i.e., excluding the final classifier layer).
- pass our images through this truncated network, obtaining feature vectors describing each image.

These feature vectors capture higher-level visual patterns (such as edges, shapes, or textures), which can then be used as input to a simpler classifier.

*Note:* Using a pre-trained CNN can vastly reduce the number of features, because these models only output a moderate number of activation maps in their final convolutional layers, rather than tens of thousands of raw pixel values.

In [ ]:
import tensorflow as tf

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Load the VGG16 model pre-trained on ImageNet
# 'include_top=False' removes the fully connected layers at the top, so we only keep the convolutional part (feature extractor)
# 'input_shape' defines the size of input images (must match IMG_SIZE and use 3 channels for RGB)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Create a new model that outputs the convolutional feature maps from the original VGG16
# We keep the same input, but output from the last convolutional layer
model = Model(inputs=base_model.input, outputs=base_model.output)

# Reshape the image data to match the model's expected input format:
# -1 lets NumPy automatically calculate the number of samples
# IMG_SIZE x IMG_SIZE defines height and width
# 3 indicates the number of colour channels (RGB)
X_reshaped = X.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# Print the shape of the reshaped input to verify dimensions
print("X_reshaped shape:", X_reshaped.shape)

# Check for any missing (NaN) values in the image data, which could cause errors during prediction
print("X_reshaped NaN values:", np.isnan(X_reshaped).sum())

# Print the minimum and maximum pixel values to check the value range
# Pre-trained models like VGG typically expect pixel values in a certain range (e.g. 0–255 or 0–1)
print("X_reshaped min/max:", X_reshaped.min(), X_reshaped.max())

# Pass the reshaped image data through the VGG16 model to extract feature maps
# This produces a compressed, abstract representation of each image, capturing key visual features
X_cnn = model.predict(X_reshaped)

# Print the shape of the extracted features to understand how the images have been transformed
print('Extracted feature shape:', X_cnn.shape)


The output provides key insights about the input data (X_reshaped) and the feature maps extracted by the VGG16 model.
```
X_reshaped shape: (21, 64, 64, 3)
```
We have 21 images, each of size 64×64 pixels with 3 color channels (RGB). And there are no missing or invalid values in the dataset.

The pixel intensity ranges from 0 to 255, which suggests that the images may not be normalised (VGG16 usually expects values in the `[0, 1]` range or preprocessed using keras.applications.vgg16.preprocess_input).
```
Extracted feature shape: (21, 2, 2, 512)
```
The output for each image is a 2×2 feature map with 512 channels (depth) per pixel. Since we used  `include_top=False`, the fully connected layers were removed, and VGG16 outputs a downsampled feature representation of the input.

### Feature map downsampling
When we pass a 64×64 image through the VGG16 model, the output is much smaller — just a 2×2 grid. This happens because the model uses layers that shrink the image size step by step. These layers are designed to zoom out gradually, so the model focuses less on exact pixel positions and more on the overall patterns.

Even though the final output is small, it is very rich in information. Instead of RGB colour channels, the output now has *512 feature channels*. Each of these captures a different type of pattern or texture the model has learned — like edges, shapes, or textures that are useful for recognising objects.

Let’s now take a look at what these features look like by plotting them:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Select the first image in the batch
image_idx = 0
feature_maps = X_cnn[image_idx]  # Shape: (2, 2, 512)

# Select a few feature maps to visualise
num_features = min(8, feature_maps.shape[-1])  # Show at most 8 feature maps
fig, axes = plt.subplots(1, num_features, figsize=(10, 4))

for i in range(num_features):
    feature_map = feature_maps[:, :, i]  # Extract i-th feature map
    axes[i].imshow(feature_map, cmap='inferno')
    axes[i].axis("off")
    axes[i].set_title(f"Feature {i+1}")

plt.suptitle("Sample Feature Maps from VGG16")
plt.show()


Each feature map captures different patterns (edges, textures, or complex features). The darker/lighter regions indicate which parts of the image activated that filter. To see which regions of the image activate the network the most, we can average the 512 feature maps and visualise the activation intensity.

Bright areas equal strongly activated regions. Whereas, dark areas are weakly activated regions. This helps understand which parts of an image VGG16 considers important.

These extracted features can be flattened further for classical machine learning algorithms, or used directly in a classifier:

In [ ]:
X_flattened = X_cnn.reshape(X_cnn.shape[0], -1)
print(X_flattened.shape)

### Saliency maps and Grad-CAM

Saliency maps and Grad-CAM are tools that help us *see* what a neural network is focusing on when it looks at an image.

When a model makes a prediction — for example, saying an image is a cat — these methods highlight the parts of the image that were most important in making that decision.

- *Saliency maps* work by measuring how much each pixel affects the model’s prediction. If changing a certain pixel would change the result, that pixel gets highlighted. This shows us which areas the model finds most "sensitive" or meaningful.

- *Grad-CAM* builds on this idea by highlighting entire regions, rather than individual pixels. It shows us which parts of the image the model is “looking at” the most, giving a more human-friendly explanation of what influenced the prediction.

In short, these techniques give us a kind of “heatmap” over the image, helping us understand why the model made its choice — a powerful way to build trust and transparency in AI systems.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load pre-trained VGG16 model
model = tf.keras.applications.VGG16(weights="imagenet")

model.trainable = False  # Keep it fixed


#### Load and preprocess the image
We write a simple helper function to load and preproces each image:

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = tf.keras.applications.vgg16.preprocess_input(img)  # Normalize like ImageNet
    return img

### Saliency map
This next function generates a **saliency map**, which visually highlights the parts of an image that most influenced a model's prediction. The process begins by preprocessing the input image—resizing it, normalising pixel values, and formatting it so the model can use it. The image is then passed into a neural network, and the model makes a prediction. 

To understand which parts of the image contributed most to this prediction, the code uses something called a *gradient*, which measures how much a small change in each pixel would affect the model’s confidence in its top predicted class. The stronger the gradient, the more important that pixel is. The function then processes these gradients, reducing them into a 2D map that highlights the most "influential" areas. 

Finally, it scales the values between 0 and 1, so the map can be visualised as a clear and interpretable heatmap:

In [ ]:
def compute_saliency_map(image_path):
    # Preprocess the image (e.g. resize, normalise, expand dims), custom function assumed
    img = preprocess_image(image_path)

    # Convert the preprocessed image to a TensorFlow tensor of type float32
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)

    # Start recording operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Tell the tape to watch the image tensor, so we can compute gradients with respect to it
        tape.watch(img_tensor)

        # Make a prediction using the model
        predictions = model(img_tensor)

        # Identify the class with the highest predicted probability
        top_class = tf.argmax(predictions[0])

        # Define the "loss" as the model’s confidence in the top class
        loss = predictions[:, top_class]

    # Compute the gradient of the top class score with respect to the input image
    grads = tape.gradient(loss, img_tensor)

    # Take the absolute value of the gradients and reduce across the colour channels (RGB) to get a single 2D map
    saliency = tf.reduce_max(tf.abs(grads), axis=-1)[0]

    # Normalise the saliency map values between 0 and 1 for visualisation
    saliency = (saliency - tf.reduce_min(saliency)) / (tf.reduce_max(saliency) - tf.reduce_min(saliency))

    # Convert the result to a NumPy array and return
    return saliency.numpy()


Lets now use this function and apply it to a sample image `dog.1.jpg`:

In [ ]:
image_path = dataset_path + "dogs/dog.1.jpg"  # Provide an image path
print(image_path)

saliency = compute_saliency_map(image_path)
original_img = cv2.imread(image_path)
original_img = cv2.resize(original_img, (224, 224))

plt.figure(figsize=(8, 4))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis("off")

# Saliency map
plt.subplot(1, 2, 2)
plt.imshow(saliency, cmap="hot")
plt.title("Saliency Map")
plt.axis("off")

This method computes gradients of the most confident class with respect to input pixels. It also, highlights important regions that strongly influence the model’s decision.

### Grad-CAM heatmap
Grad-CAM (Gradient-weighted Class Activation Mapping) produces a heatmap over the image, highlighting regions that strongly influence the model's output. 

It is particularly helpful for CNNs, where convolutional layers learn spatially meaningful patterns.

#### Load VGG16 model (without top layers)

We use the VGG16 model, a well-known deep learning architecture trained on millions of images from the ImageNet dataset. Loading the model without the top (fully connected) layers, keeps just the convolutional part of the model. 

This part acts as a powerful *feature extractor*, capturing useful visual patterns like edges, textures, and shapes from the input images. Instead of making predictions, the model outputs these extracted features, which we can then use for tasks like image classification, clustering, or further analysis. 

This approach saves time and resources, as we benefit from knowledge the model has already learned:

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

model = tf.keras.applications.VGG16(weights="imagenet")
model.trainable = False

# Select last convolutional layer for Grad-CAM
last_conv_layer_name = "block5_conv3"


#### Compute Grad-CAM heatmap
The *Grad-CAM (Gradient-weighted Class Activation Mapping)* approach is a popular technique for visually explaining what a deep learning model is focusing on when it makes a prediction.

The method works by identifying which parts of an image contributed most to the model’s decision. It does this by examining the *gradients* flowing back from the model’s output to a specific *convolutional layer*. These gradients are used to calculate the importance of each feature map (or channel) in that layer.

Once the important channels are identified, the function combines them to create a *heatmap* that highlights the regions in the image that had the most influence on the predicted class. This heatmap is then normalised for visual clarity, allowing us to overlay it on the original image and see where the model was “looking” when it made its decision.

In short, this approach helps us *interpret* deep learning models by revealing which visual features drove a particular classification:

In [ ]:
def compute_gradcam(image_path, model, layer_name):
    # Preprocess the input image (e.g., resize, normalise, expand dims)
    img = preprocess_image(image_path)

    # Create a modified model that outputs both:
    # 1) The activation from the specified convolutional layer
    # 2) The final prediction from the original model
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    # Record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Forward pass through the model: get both the conv layer output and final prediction
        conv_output, predictions = grad_model(img)

        # Identify the class with the highest predicted probability
        top_class = tf.argmax(predictions[0])

        # Define the loss as the model’s confidence in the top class
        loss = predictions[:, top_class]

    # Compute the gradient of the class score with respect to the conv layer output
    grads = tape.gradient(loss, conv_output)

    # Global average pooling: average the gradients over height and width dimensions
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Remove the batch dimension from the conv layer output
    conv_output = conv_output[0]

    # Multiply each channel in the conv layer by its corresponding pooled gradient (i.e., its importance)
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)

    # Apply ReLU to keep only positive values (areas that positively influence the class score)
    heatmap = np.maximum(heatmap, 0)

    # Normalise the heatmap values to range from 0 to 1
    heatmap /= np.max(heatmap)

    # Return the heatmap as a NumPy array
    return heatmap


Now we can generate a Grad-CAM heatmap, let's overlay it onto an original image, to make things flexible, we will create another function to do this `overlay_heatmap`:

In [ ]:
def overlay_heatmap(image_path, heatmap, alpha=0.4):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))

    # Resize heatmap to match image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Convert heatmap to color
    heatmap = np.uint8(255 * heatmap)  # Scale to 0-255
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply colour map

    # Overlay heatmap on original image
    superimposed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)

    # Display original and heatmap images
    plt.figure(figsize=(8, 4))

    # Original image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")

    # Grad-CAM overlay
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title("Grad-CAM Heatmap")
    plt.axis("off")

Now we can execute both `compute_gradcam`, and `overlay_heatmp` on an image of our choice:

In [ ]:
image_path = dataset_path + "dogs/dog.1.jpg"  # Provide an image path

heatmap = compute_gradcam(image_path, model, last_conv_layer_name)

overlay_heatmap(image_path, heatmap)

While these methods are not strictly feature selection techniques in the traditional sense, they are useful for understanding and manually selecting important image regions, thereby potentially reducing unnecessary features or focusing on the most relevant parts of an image.

### Entropy-based feature selection

Entropy is a measure of uncertainty or unpredictability within data. In our case, it is a measure of visual complexity or texture within an image and can be used to identify informative regions.

This approach allows us to represent each image numerically based on its *textural content*, which can be useful for image classification or pattern analysis tasks.

A pixel with low variance or repetitive values across many images may provide very little information to distinguish between classes. By contrast, pixels or regions with high entropy might capture essential variations.

1. We convert the image to greyscale.
2. We compute the local entropy of each pixel, often using a small neighbourhood (for instance, using a disk-shaped structuring element as provided by `skimage.morphology.disk`).
3. We can then selectively retain pixels or regions that have higher entropy values.

Below, we illustrate how to compute the entropy for each pixel using `skimage`. In a real-world scenario, you might:
- Convert these per-pixel entropy values into a single statistic per region.
- Select the top x% of pixels with the highest entropy.
- Or threshold the entropy to keep only sufficiently informative pixels.

Because this approach still involves per-pixel computations, it might be slow or unwieldy for large datasets. Nonetheless, it is a straightforward and interpretable starting point.

Again, we create our own function `compute_entropy` to easily apply it to any image in our collection:

In [ ]:
import cv2
import glob
import numpy as np
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.transform import resize

IMG_SIZE = 128  # Define a fixed image size to standardise input

def compute_entropy(img_path):
    # Read the image from the given file path
    img = cv2.imread(img_path)

    # Check if the image was successfully loaded
    if img is None:
        print(f"Error: Unable to read image {img_path}")
        return None

    # Convert the image to grayscale since entropy is computed on intensity values
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Resize the image to a consistent size (128x128)
    # This ensures all output feature vectors are the same length
    grayscale = cv2.resize(grayscale, (IMG_SIZE, IMG_SIZE))

    # Ensure the image data type is 8-bit unsigned integers (required by skimage's entropy function)
    grayscale = grayscale.astype(np.uint8)

    # Compute local entropy using a circular neighbourhood with radius 5 pixels
    ent = entropy(grayscale, disk(5))

    # Flatten the 2D entropy matrix into a 1D array to use as a feature vector
    return ent.flatten()


Now we can extract *entropy features* from all cat images in the dataset. 

To do this, our code will perform the following steps:

- *Collect file paths*: We use `glob` to gather all `.jpg` files from the `cats` folder.  
- *Loop through each image*: For each image, we apply the `compute_entropy()` function to calculate the local entropy values across the image.  
- *Store features*: The resulting values (flattened into a 1D feature vector) are added to a list.  
- *Convert to NumPy array*: This list is then converted into an array for use in modelling or analysis.  
- *Inspect the result*: We print the shape of the final array to confirm the number of images and the length of each feature vector.

Let's run this next cell, and see the result:

In [ ]:
# Get all cat image paths
cat_files = glob.glob(dataset_path + '/cats/*.jpg')

X_entropy_cats = []

for f in cat_files:
    ent_vals = compute_entropy(f)
    X_entropy_cats.append(ent_vals)

X_entropy_cats = np.array(X_entropy_cats)
print('Shape of entropy features for cats:', X_entropy_cats.shape)

Since we are working with images, it makes sense to plot them for inspection:

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Select one image and compute its entropy
sample_image = cat_files[0]  # Pick the first image
entropy_map = compute_entropy(sample_image)

# Reshape the entropy map back to 2D (assuming IMG_SIZE x IMG_SIZE)
entropy_map_reshaped = entropy_map.reshape(IMG_SIZE, IMG_SIZE)

# Load and resize the original image for display
original_image = cv2.imread(sample_image)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
original_image_resized = cv2.resize(original_image, (IMG_SIZE, IMG_SIZE))

# Plot original image and entropy map side by side
plt.figure(figsize=(10, 4))

# Show original image
plt.subplot(1, 2, 1)
plt.imshow(original_image_resized)
plt.title("Original Image")
plt.axis("off")

# Show entropy map
plt.subplot(1, 2, 2)
plt.imshow(entropy_map_reshaped, cmap='inferno')
plt.colorbar(label="Entropy")
plt.title("Entropy Map")
plt.axis("off")

plt.tight_layout()
plt.show()


In practice, you might combine cats and dogs, then label them accordingly. Then, you could decide on an entropy threshold or a selection strategy.

### Dimensionality reduction with Autoencoders

Autoencoders are a type of neural network that learn how to *compress* and then *rebuild* data. The goal is to take something complex — like an image — and turn it into a much simpler version that still keeps all the important information.

The model has two main parts:

- *Encoder*: This part squeezes the original image into a smaller, more compact form, removing unnecessary details and keeping only the most useful features.  
- *Decoder*: This part tries to recreate the original image from that compact version as accurately as possible.

During training, the model learns which parts of the image are essential and which parts can be ignored. Once trained, we can use just the encoder to reduce the size of our data. This gives us a set of features that are much smaller in size, but still meaningful.

This process is known as *dimensionality reduction*. It works in a similar way to methods like PCA (Principal Component Analysis), but autoencoders are more powerful because they can learn complex, non-linear patterns.

Below, we prepare the data and build the autoencoder model.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Define parameters
IMG_SIZE = 64  # Resize all images to 64x64
BATCH_SIZE = 8  # Reduce batch size if dataset is small

# Load Images from Directory
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Normalise images (Scale pixel values to [0,1]) and keep labels to prevent errors
train_ds = train_ds.map(lambda x, _: (x / 255.0, x / 255.0))

# Debug: Check if dataset is loading correctly
for batch in train_ds.take(1):
    print("Sample batch shape:", batch[0].shape)

The values returned as the image dataset shape e.g. `(8, 64, 64, 3)`, describe the structure of our processed image data in numerical terms. Here’s what each number means:

- 8– The total number of images in the batch of training

- 64 – The height of each image in pixels (after resizing)

- 64 – The width of each image in pixels

- 3 – The number of colour channels (Red, Green, Blue)

So this tells us that we have 8 colour images in our batch, each resized to 64×64 pixels, ready for model training or further analysis.

In our case, let's set up and train the autoencoder:

In [ ]:
# Define the Autoencoder architecture
# Set the input image shape (e.g. 128x128 pixels with 3 colour channels)
input_shape = (IMG_SIZE, IMG_SIZE, 3)

# Set the size of the encoded representation (latent space)
encoding_dim = 256

# Encoder: Compresses the input image into a smaller feature representation
input_img = layers.Input(shape=input_shape)  # Input layer

# First convolutional layer: 32 filters, ReLU activation
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)

# Downsample the feature map (reduce spatial dimensions while keeping information)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

# Second convolutional layer: 64 filters, ReLU activation
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

# Another downsampling step
x = layers.MaxPooling2D((2, 2), padding='same')(x)

# Final encoding layer: compress the feature maps into a more compact representation
encoded = layers.Conv2D(encoding_dim, (3, 3), activation='relu', padding='same')(x)

# Decoder: Reconstructs the original image from the encoded representation
# First decoding layer: attempt to reconstruct the 64-filter feature map
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)

# Upsample to increase spatial resolution
x = layers.UpSampling2D((2, 2))(x)

# Second decoding layer: return to 32 filters
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)

# Final upsampling step to reach original size
x = layers.UpSampling2D((2, 2))(x)

# Output layer: 3 filters to match RGB image channels, sigmoid for pixel values between 0 and 1
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Compile the autoencoder model
# Connect input and output through the defined layers
autoencoder = models.Model(input_img, decoded)

# Compile the model using the Adam optimiser and mean squared error (MSE) loss
# MSE measures how close the reconstructed image is to the original
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
# Fit the model to the training dataset (train_ds) for 10 epochs
autoencoder.fit(train_ds, epochs=10)

Once our model has finished training we can use it to create reconstructed version of the original image to use as features in simpler model:

In [ ]:
# Get some test images from the dataset
test_images, _ = next(iter(train_ds))  # Get a batch of test images

# Pass them through the autoencoder
reconstructed_images = autoencoder.predict(test_images)

# Plot Original vs Reconstructed Images
fig, axes = plt.subplots(2, BATCH_SIZE, figsize=(12, 4))  # 2 rows, 10 columns

for i in range(min(BATCH_SIZE, test_images.shape[0])):  # Ensure we don't index out of range
    # Original images
    axes[0, i].imshow(test_images[i])
    axes[0, i].axis('off')

    # Reconstructed images
    axes[1, i].imshow(reconstructed_images[i])
    axes[1, i].axis('off')

# Add titles
axes[0, 0].set_title('Original Images')
axes[1, 0].set_title('Reconstructed Images')

### What have we learnt?

When working with image data, choosing or extracting the right features can have a major impact on model performance and efficiency. Many classical feature selection methods do not directly account for the spatial structure of images, so more sophisticated approaches like CNN-based feature extraction, saliency maps, and deep learning–based dimensionality reduction are often more suitable.  In summary:

- *Pre-Trained CNNs*: are a strong, practical baseline for feature extraction, making use of knowledge gained from large datasets.
- *Saliency Maps and Grad-CAM*: methods help us visualise which parts of the image matter most for a model's classification, aiding interpretability and manual selection of relevant regions.
- *Entropy*: can help locate highly variable regions in images, which might contain more discriminative information than homogeneous areas.
- *Mutual Information*: allows us to rank each pixel's relevance to the class label, though it ignores spatial dependencies and can be computationally demanding.
- *Autoencoders*: learn compressed representations tailored to the image data (particularly convolutional autoencoders), providing an alternative to more linear dimensionality reduction methods .

In practice, one often combines or compares multiple approaches: for instance, extracting CNN-based features and then using a technique like mutual information or entropy to further refine which features are retained, or applying an autoencoder to reduce dimensionality and then training a relatively simple classifier on the lower-dimensional representation.

Experimentation and careful assessment (e.g., via cross-validation) will guide which method (or combination of methods) provides the most reliable, accurate, and interpretable results for a specific image classification task.